In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
# spark-3.3.2/ 
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [31.2 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,069 kB]
Get:14 http://arc

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
homesales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
homesales_df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
homesales_df.createOrReplaceTempView('home_sales')
# Create a temporary table called home_sales.

In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

query = """
SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR(date)
"""

# Execute the SQL query and store the results in a DataFrame
result_df = spark.sql(query)

# Show the results
result_df.show()


+----+---------+
|year|avg_price|
+----+---------+
|2022|296363.88|
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
+----+---------+



In [6]:
# 4. 
query2 = """
SELECT year(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY year(date_built)
"""

# Execute the SQL query and store the results in a DataFrame
result2_df = spark.sql(query2)

# Show the results
result2_df.show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2015| 288770.3|
|      2013|295962.27|
|      2014|290852.27|
|      2012|293683.19|
|      2016|290555.07|
|      2010|292859.62|
|      2011|291117.47|
|      2017|292676.79|
+----------+---------+



In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

query3 = """
SELECT year(date_built) AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY year(date_built)
"""

# Execute the SQL query and store the results in a DataFrame
result3_df = spark.sql(query3)

# Show the results
result3_df.show()



+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2015|297609.97|
|      2013|303676.79|
|      2014|298264.72|
|      2012|307539.97|
|      2016| 293965.1|
|      2010|285010.22|
|      2011|276553.81|
|      2017|280317.58|
+----------+---------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

query4 = """
SELECT AVG(view) AS avg_view
FROM home_sales
WHERE price >= 350000
"""

result4_df = spark.sql(query4)

result4_df.show()

start_time = time.time()


print("Query runtime: %s seconds" % (time.time() - start_time))

#print("--- %s seconds ---" % (time.time() - start_time))

+--------+
|avg_view|
+--------+
|  32.264|
+--------+

Query runtime: 0.00010395050048828125 seconds


In [9]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price of greater than $350,000. 
# Determine the runtime and compare it to uncached runtime.
query5 = """
SELECT AVG(view) AS avg_view
FROM home_sales
WHERE price <= 350000
"""
result5_df = spark.sql(query5)

# Show the results
result5_df.show()


start_time = time.time()

print("Query runtime with caching: %s seconds" % (time.time() - start_time))



+------------------+
|          avg_view|
+------------------+
|24.999232921216496|
+------------------+

Query runtime with caching: 0.00011968612670898438 seconds


In [12]:
# 10. Partition the home sales dataset by the date_built field.
homesales_df.write.partitionBy("date_built").parquet("homesales_partitioned")

In [13]:
# 11. Read the formatted parquet data.
partitioned_df = spark.read.parquet("homesales_partitioned")

partitioned_df.show(5)

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
+--------------------+----------+------+--------+---------+-----------+--------+

In [15]:
# 12. Create a temporary table for the parquet data.
partitioned_df.createOrReplaceTempView("homesales_partitioned")

spark.sql("SELECT * FROM homesales_partitioned").show(5)

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
+--------------------+----------+------+--------+---------+-----------+--------+

In [16]:
# 13. Run the query that filters out the view ratings with average price of greater than $350,000 with the parquet DataFrame. 
# Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 
parquet_result = spark.sql("SELECT view, ROUND(AVG(price), 2) AS avg_price FROM homesales_partitioned WHERE price >= 350000 AND bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 GROUP BY view")

# Show the query result
parquet_result.show()

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|   7| 398251.4|
|  15| 402942.0|
|  54| 596768.0|
|  11| 396833.5|
|  29| 394046.1|
|  42|430864.33|
|   3|402697.25|
|  30|413549.25|
|  34|403201.33|
|  59| 758574.0|
|   8| 405157.0|
|  22|419964.67|
|  28|398963.67|
|  35|380399.25|
|  16| 389990.5|
|  52| 568309.0|
|   0| 397933.1|
|  71| 589309.0|
|  47| 383360.5|
|  99| 840349.0|
+----+---------+
only showing top 20 rows

--- 0.0001289844512939453 seconds ---


In [17]:
# Runtime here was 0.000129 seconds, 
#compared to the above of 0.000120 seconds with caching and 
#0.000104 seconds pre-caching.

In [18]:
# 14. Uncache the home_sales temporary table.
spark.catalog.cacheTable("home_sales")

In [19]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


True